In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


This notebook is a refactored version of a Multiscale Convolutional Network solution for NeuroHack at Yandex

In [3]:
import random

from tqdm import tqdm
import pandas as pd
import h5py
import numpy as np
from scipy.signal import resample

In [4]:
train_path = "train.h5"
test_path = "test.h5"
model_dump_path = "convnet-multiscale-true-01988"

slice_len = 1125

Read train data into a dict of (subject_id, (X, y)) pairs

In [6]:
subjects = {}
with h5py.File(train_path, "r") as data_file:
    for subject, subject_data in data_file.items():
        X = subject_data["data"][:]
        y = subject_data["labels"][:][0]
        subjects[subject] = (X, y)

We need to split our data into train and local validation sets. For local validation we select random slices of slice_len. They may overlap with slices in train set, this split is not optimal.

In [7]:
from sklearn.model_selection import train_test_split

def train_val_split(X, y):
    start_indices = list(range(0, len(X) - slice_len))
    y = y[:len(start_indices)]
    indices_train, indices_test, _, _ = train_test_split(start_indices, y)
    return {"train_ind": indices_train, "val_ind": indices_test, "X": X, "y": y}

In [8]:
for subject in subjects:
    X, y = subjects[subject][0], subjects[subject][1]
    X = X.T
    subjects[subject] = train_val_split(X, y)

Define utility function to convert class labels to one-hot encoding

In [9]:
def to_onehot(y):
    onehot = np.zeros(3)
    onehot[y] = 1
    return onehot

Define a funtion that will select random subject and find a random subsequence of consistent class

In [10]:
def generate_slice(slice_len, val=False):
    subject_data = random.choice(list(subjects.values()))
    if val is True:
        indices, y, X = subject_data["val_ind"], subject_data["y"], subject_data["X"]
    else:
        indices, y, X = subject_data["train_ind"], subject_data["y"], subject_data["X"]
    
    while True:
        slice_start = random.choice(indices)
        slice_end = slice_start + slice_len
        slice_x = X[slice_start:slice_end]
        slice_y = y[slice_start:slice_end]
        
        if len(set(slice_y)) == 1:
            return slice_x, to_onehot(slice_y)

Define a generator that will yield batches of resampled input time series and according class labels in infinite loop

In [11]:
def data_generator(batch_size, slice_len, val=False):
    while True:
        batch_x = []
        batch_y = []
        
        for i in range(0, batch_size):
            x, y = generate_slice(slice_len, val=val)
            batch_x.append(x)
            batch_y.append(y)
            
        y = np.array(batch_y)
        
        x_256 = np.array([resample(i, 256) for i in batch_x])
        x_500 = np.array([resample(i, 500) for i in batch_x])
        x = np.array([i for i in batch_x])
        yield ([x_256, x_500, x], y)

Lets build a neural network. Import all needed layers and keras utilities

In [12]:
from keras.layers import Convolution1D, Dense, Dropout, Input, merge, GlobalMaxPooling1D
from keras.models import Model, load_model
from keras.optimizers import RMSprop

This function builds a base neural net model that performs feature extraction

In [13]:
def get_base_model(input_len, fsize):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input_seq = Input(shape=(input_len, 24))
    nb_filters = 150
    convolved = Convolution1D(nb_filters, fsize, border_mode="same", activation="tanh")(input_seq)
    processed = GlobalMaxPooling1D()(convolved)
    compressed = Dense(150, activation="tanh")(processed)
    compressed = Dropout(0.3)(compressed)
    compressed = Dense(150, activation="tanh")(compressed)
    model = Model(input=input_seq, output=compressed)            
    return model

Build and compile a graph with 3 inputs and one output

In [14]:
input256_seq = Input(shape=(256, 24))
input500_seq = Input(shape=(500, 24))
input1125_seq = Input(shape=(1125, 24))
    
base_network256 = get_base_model(256, 4)
base_network500 = get_base_model(500, 7)
base_network1125 = get_base_model(1125, 10)

embedding_256 = base_network256(input256_seq)
embedding_500 = base_network500(input500_seq)
embedding_1125 = base_network1125(input1125_seq)
    
merged = merge([embedding_256, embedding_500, embedding_1125], mode="concat")
out = Dense(3, activation='softmax')(merged)
    
model = Model(input=[input256_seq, input500_seq, input1125_seq], output=out)
    
opt = RMSprop(lr=0.005, clipvalue=10**6)
model.compile(loss="categorical_crossentropy", optimizer=opt)

C:\Users\voxxys\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(150, 4, activation="tanh", padding="same")`
  
C:\Users\voxxys\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\voxxys\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(150, 7, activation="tanh", padding="same")`
  
C:\Users\voxxys\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(150, 10, activation="tanh", padding="same")`
  
C:\Users\voxxys\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, 

In [18]:
model.save('my_model.h5')

In [15]:
model = load_model('my_model_trained_2.h5')

C:\Users\voxxys\Anaconda3\lib\site-packages\keras\engine\topology.py:1242: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [23]:
from keras.callbacks import EarlyStopping
import tensorflow as tf

nb_epoch = 5
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
samples_per_epoch = 4000
    


This cell will train the model from scratch, lets load it from the model dump instead (take a look at next code cell)!

In [ ]:
for i in range(10):
    
    print(i);

    with tf.device('/gpu:0'):
        model.fit_generator(data_generator(batch_size=50, slice_len=slice_len), samples_per_epoch, nb_epoch, 
                            callbacks=[earlyStopping], verbose=2, nb_val_samples=2000,
                            validation_data=data_generator(batch_size=50, slice_len=slice_len, val=True))
        model.save('my_model_trained_'+str(i)+'.h5');

0


C:\Users\voxxys\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 4000, 5, callbacks=[<keras.ca..., verbose=1, validation_data=<generator..., validation_steps=2000)`
  


Epoch 1/5


 314/4000 [=>............................] - ETA: 1836s - loss: 6.08 - ETA: 1038s - loss: 7.08 - ETA: 779s - loss: 6.2618 - ETA: 666s - loss: 6.349 - ETA: 604s - loss: 6.197 - ETA: 569s - loss: 6.254 - ETA: 541s - loss: 6.125 - ETA: 521s - loss: 6.039 - ETA: 502s - loss: 5.941 - ETA: 521s - loss: 5.920 - ETA: 543s - loss: 5.781 - ETA: 538s - loss: 5.803 - ETA: 521s - loss: 5.854 - ETA: 506s - loss: 5.808 - ETA: 492s - loss: 5.813 - ETA: 479s - loss: 6.048 - ETA: 470s - loss: 6.028 - ETA: 463s - loss: 5.997 - ETA: 456s - loss: 5.922 - ETA: 450s - loss: 5.952 - ETA: 444s - loss: 5.990 - ETA: 438s - loss: 6.004 - ETA: 433s - loss: 6.002 - ETA: 426s - loss: 5.986 - ETA: 423s - loss: 6.000 - ETA: 418s - loss: 5.940 - ETA: 414s - loss: 5.905 - ETA: 409s - loss: 5.852 - ETA: 406s - loss: 5.889 - ETA: 402s - loss: 5.895 - ETA: 398s - loss: 5.930 - ETA: 394s - loss: 6.021 - ETA: 392s - loss: 6.030 - ETA: 388s - loss: 6.017 - ETA: 386s - loss: 5.997 - ETA: 383s - loss: 5.988 - ETA: 381s - loss: 

 628/4000 [===>..........................] - ETA: 255s - loss: 5.895 - ETA: 255s - loss: 5.894 - ETA: 255s - loss: 5.898 - ETA: 255s - loss: 5.899 - ETA: 255s - loss: 5.899 - ETA: 255s - loss: 5.899 - ETA: 255s - loss: 5.901 - ETA: 254s - loss: 5.906 - ETA: 254s - loss: 5.907 - ETA: 254s - loss: 5.906 - ETA: 254s - loss: 5.906 - ETA: 254s - loss: 5.902 - ETA: 254s - loss: 5.903 - ETA: 254s - loss: 5.900 - ETA: 254s - loss: 5.901 - ETA: 254s - loss: 5.900 - ETA: 254s - loss: 5.898 - ETA: 253s - loss: 5.897 - ETA: 253s - loss: 5.896 - ETA: 253s - loss: 5.897 - ETA: 253s - loss: 5.901 - ETA: 253s - loss: 5.899 - ETA: 253s - loss: 5.897 - ETA: 253s - loss: 5.900 - ETA: 253s - loss: 5.899 - ETA: 253s - loss: 5.898 - ETA: 252s - loss: 5.893 - ETA: 252s - loss: 5.896 - ETA: 252s - loss: 5.893 - ETA: 252s - loss: 5.902 - ETA: 252s - loss: 5.906 - ETA: 252s - loss: 5.903 - ETA: 252s - loss: 5.902 - ETA: 251s - loss: 5.902 - ETA: 251s - loss: 5.900 - ETA: 251s - loss: 5.898 - ETA: 251s - loss: 5

...

1570/4000 [==========>...................] - ETA: 173s - loss: 5.884 - ETA: 173s - loss: 5.883 - ETA: 173s - loss: 5.884 - ETA: 173s - loss: 5.883 - ETA: 173s - loss: 5.884 - ETA: 173s - loss: 5.884 - ETA: 173s - loss: 5.885 - ETA: 173s - loss: 5.885 - ETA: 172s - loss: 5.884 - ETA: 172s - loss: 5.883 - ETA: 172s - loss: 5.883 - ETA: 172s - loss: 5.885 - ETA: 172s - loss: 5.886 - ETA: 172s - loss: 5.886 - ETA: 172s - loss: 5.887 - ETA: 172s - loss: 5.886 - ETA: 172s - loss: 5.885 - ETA: 172s - loss: 5.886 - ETA: 172s - loss: 5.887 - ETA: 172s - loss: 5.888 - ETA: 172s - loss: 5.887 - ETA: 172s - loss: 5.888 - ETA: 172s - loss: 5.887 - ETA: 172s - loss: 5.887 - ETA: 172s - loss: 5.887 - ETA: 171s - loss: 5.887 - ETA: 171s - loss: 5.886 - ETA: 171s - loss: 5.886 - ETA: 171s - loss: 5.886 - ETA: 171s - loss: 5.886 - ETA: 171s - loss: 5.886 - ETA: 171s - loss: 5.884 - ETA: 171s - loss: 5.884 - ETA: 171s - loss: 5.885 - ETA: 171s - loss: 5.885 - ETA: 171s - loss: 5.885 - ETA: 171s - loss: 5

1884/4000 [=============>................] - ETA: 153s - loss: 5.890 - ETA: 153s - loss: 5.889 - ETA: 153s - loss: 5.889 - ETA: 153s - loss: 5.889 - ETA: 153s - loss: 5.889 - ETA: 153s - loss: 5.889 - ETA: 153s - loss: 5.889 - ETA: 153s - loss: 5.889 - ETA: 152s - loss: 5.890 - ETA: 152s - loss: 5.890 - ETA: 152s - loss: 5.889 - ETA: 152s - loss: 5.890 - ETA: 152s - loss: 5.888 - ETA: 152s - loss: 5.888 - ETA: 152s - loss: 5.888 - ETA: 152s - loss: 5.888 - ETA: 152s - loss: 5.888 - ETA: 152s - loss: 5.888 - ETA: 152s - loss: 5.887 - ETA: 152s - loss: 5.889 - ETA: 152s - loss: 5.887 - ETA: 152s - loss: 5.887 - ETA: 152s - loss: 5.887 - ETA: 151s - loss: 5.887 - ETA: 151s - loss: 5.887 - ETA: 151s - loss: 5.887 - ETA: 151s - loss: 5.888 - ETA: 151s - loss: 5.888 - ETA: 151s - loss: 5.889 - ETA: 151s - loss: 5.890 - ETA: 151s - loss: 5.889 - ETA: 151s - loss: 5.889 - ETA: 151s - loss: 5.889 - ETA: 151s - loss: 5.888 - ETA: 151s - loss: 5.888 - ETA: 151s - loss: 5.888 - ETA: 151s - loss: 5

2198/4000 [===============>..............] - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.894 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.892 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.892 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 133s - loss: 5.893 - ETA: 132s - loss: 5.893 - ETA: 132s - loss: 5.893 - ETA: 132s - loss: 5.892 - ETA: 132s - loss: 5.892 - ETA: 132s - loss: 5.892 - ETA: 132s - loss: 5.892 - ETA: 132s - loss: 5.893 - ETA: 132s - loss: 5.893 - ETA: 132s - loss: 5.894 - ETA: 132s - loss: 5.894 - ETA: 132s - loss: 5.894 - ETA: 132s - loss: 5.895 - ETA: 132s - loss: 5.896 - ETA: 132s - loss: 5.897 - ETA: 132s - loss: 5.896 - ETA: 132s - loss: 5.896 - ETA: 131s - loss: 5.897 - ETA: 131s - loss: 5.896 - ETA: 131s - loss: 5.895 - ETA: 131s - loss: 5.894 - ETA: 131s - loss: 5.894 - ETA: 131s - loss: 5

2520/4000 [=================>............] - ETA: 113s - loss: 5.890 - ETA: 113s - loss: 5.891 - ETA: 113s - loss: 5.891 - ETA: 113s - loss: 5.890 - ETA: 113s - loss: 5.890 - ETA: 113s - loss: 5.889 - ETA: 113s - loss: 5.889 - ETA: 113s - loss: 5.889 - ETA: 113s - loss: 5.889 - ETA: 113s - loss: 5.889 - ETA: 113s - loss: 5.890 - ETA: 113s - loss: 5.890 - ETA: 113s - loss: 5.891 - ETA: 112s - loss: 5.890 - ETA: 112s - loss: 5.890 - ETA: 112s - loss: 5.890 - ETA: 112s - loss: 5.890 - ETA: 112s - loss: 5.890 - ETA: 112s - loss: 5.891 - ETA: 112s - loss: 5.891 - ETA: 112s - loss: 5.891 - ETA: 112s - loss: 5.891 - ETA: 112s - loss: 5.891 - ETA: 112s - loss: 5.892 - ETA: 112s - loss: 5.892 - ETA: 112s - loss: 5.892 - ETA: 112s - loss: 5.892 - ETA: 112s - loss: 5.892 - ETA: 112s - loss: 5.892 - ETA: 111s - loss: 5.891 - ETA: 111s - loss: 5.891 - ETA: 111s - loss: 5.892 - ETA: 111s - loss: 5.891 - ETA: 111s - loss: 5.891 - ETA: 111s - loss: 5.891 - ETA: 111s - loss: 5.891 - ETA: 111s - loss: 5

2779/4000 [===================>..........] - ETA: 93s - loss: 5.89 - ETA: 93s - loss: 5.89 - ETA: 93s - loss: 5.89 - ETA: 93s - loss: 5.89 - ETA: 93s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 92s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.88 - ETA: 91s - loss: 5.88 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.89 - ETA: 91s - loss: 5.88 - ETA: 91s - loss: 5.88 - ETA: 91s - loss: 5.88 - ETA: 91s - loss: 5.88 - ETA: 91s - loss: 5.88 - ETA: 90s - loss: 5.88 - ETA: 90s - loss: 5.88 - ETA: 90s - loss: 5.

In [20]:
model.save('my_model_trained.h5')

In [40]:
model = load_model('convnet-multiscale-true-01988')

C:\Users\voxxys\Anaconda3\lib\site-packages\keras\engine\topology.py:1242: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(batch_input_shape=[None, 256..., sparse=False, name="input_7", dtype="float32")`
  return cls(**config)
C:\Users\voxxys\Anaconda3\lib\site-packages\keras\engine\topology.py:1242: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(batch_input_shape=[None, 500..., sparse=False, name="input_8", dtype="float32")`
  return cls(**config)
C:\Users\voxxys\Anaconda3\lib\site-packages\keras\engine\topology.py:1242: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(batch_input_shape=[None, 112..., sparse=False, name="input_9", dtype="float32")`
  return cls(**config)
C:\Users\voxxys\Anaconda3\lib\site-packages\keras\engine\topology.py:1242: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(batch_input_shape=[None, 256..., sparse=False, name="input_10", dtype="float32")`
  retu

ValueError: Shapes must be equal rank, but are 3 and 4 for 'Assign_42' (op: 'Assign') with input shapes: [4,24,100], [4,1,24,100].

In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 24)       0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 500, 24)       0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 1125, 24)      0                                            
____________________________________________________________________________________________________
model_1 (Model)                  (None, 150)           59850       input_1[0][0]                    
___________________________________________________________________________________________

Read test data into a nested structure with multiple chunks for each subject

In [18]:
with h5py.File("test.h5", "r") as data_file:
    test = {}
    for subject, subject_data in data_file.items():
        test[subject] = {}
        for chunk_id, chunk in data_file[subject].items():
            test[subject][chunk_id] = chunk[:]

In [19]:
test['subject_0']['chunk_0'].shape

(24, 1125)

In [20]:
# utility function that performs resampling of input timeseries 
def multiscale(chunk):
    resampled_256 = resample(chunk, 256)
    resampled_500 = resample(chunk, 500)
    return [resampled_256, resampled_500, chunk]

Make prediction for test data

In [21]:
df = []
for subj in test:
    for chunk in tqdm(test[subj]):
        data = {}
        data["subject_id"] = int(subj.split("_")[-1])
        data["chunk_id"] = int(chunk.split("_")[-1])
        arr = test[subj][chunk].T
        preds = model.predict([np.array([i]) for i in multiscale(arr)])[0]
        data["class_0_score"] = preds[0]
        data["class_1_score"] = preds[1]
        data["class_2_score"] = preds[2]
        for i in range(0, 1125):
            data["tick"] = i
            df.append(data.copy())
df = pd.DataFrame(df)
df = df[["subject_id", "chunk_id", "tick", "class_0_score",
         "class_1_score","class_2_score"]]

100%|█████████████████████████████████████████| 89/89 [00:00<00:00, 235.28it/s]


In [22]:
df.head()

,subject_id,chunk_id,tick,class_0_score,class_1_score,class_2_score
0,0,0,0,0.11693,0.88307,3.050653e-13
1,0,0,1,0.11693,0.88307,3.050653e-13
2,0,0,2,0.11693,0.88307,3.050653e-13
3,0,0,3,0.11693,0.88307,3.050653e-13
4,0,0,4,0.11693,0.88307,3.050653e-13


In [ ]:
# save submission to .csv
df.to_csv("submission.csv")